<a href="https://colab.research.google.com/github/eebjwa/SENSE_week2_team4/blob/main/w2_proj_barcel_pah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import folium
#!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10114 sha256=a1fed5c252bfe4358c2eeb998561c59cda889d19c490d13656a83b5ed99f3e23
  Stored in directory: /root/.cache/pip/wheels/c5/07/67/5fa6e7271b46bbe0acafdc7105bbee27a39ab7132d251d822d
Successfully built geehydro


In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=_gxVliRNrYpabQ5SoWRvpcCbL3kfm6HtniZr9Vh89w8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AfDhmriLzBl7WHQ9c-u_PLNwhBeXLuhpdoDiRbuW3MScQ5eVixDVOfwbEP8

Successfully saved authorization token.


In [40]:
# Mosaic the layers and display 
# Load the least cloudy image of 2015 image in Acra (Ghana)
# Image near Acra (Ghana)
first = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(ee.Geometry.Point(2.18, 41.4)).filterDate('2019-03-09', '2019-03-10').sort('CLOUDY_PIXEL_PERCENTAGE').first()      
next = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(ee.Geometry.Point(2.18, 41.4)).filterDate('2019-03-09', '2019-03-10').sort('CLOUDY_PIXEL_PERCENTAGE').first()    

mosaic = ee.ImageCollection([first, next]).mosaic()


corine = ee.ImageCollection("COPERNICUS/CORINE/V20/100m").filterBounds(ee.Geometry.Point(2.18, 41.4)).filterDate('2019-03-09', '2019-03-10').first()
#corine_landcov = corine.select('landcover')

In [48]:

roi = ee.Geometry.Rectangle([ 2.16318821,41.39104253,2.19564576,41.40965281])

map = folium.Map(location=[41.4003477,2.17941699],zoom_start=14.9)
map.addLayer(mosaic.clip(roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'sen2')
#map.addLayer(corine_landcov.select('landcover'), {}, 'corine')
folium.LayerControl().add_to(map)
#map.setOptions('HYBRID') # To see GE map underneath

map.add_child(folium.LatLngPopup())

map


In [50]:
#Load in Sentinel 2 image of city n of Barcelona ()
S2 = ee.ImageCollection("COPERNICUS/S2_SR")
training_image = S2.filterBounds(ee.Geometry.Point(-0.892, 41.650)).sort('CLOUDY_PIXEL_PERCENTAGE').first()

roi = ee.Geometry.Rectangle([2.16318821,41.39104253,2.19564576,41.40965281])

map = folium.Map(location=[41.6899,0.4561],zoom_start=8)
map.addLayer(mosaic.clip(roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'sen2')
map.addLayer(training_image,{'min':0,'max':2000,'bands':['B4','B3','B2']},'train_img')
#map.addLayer(corine_landcov.select('landcover'), {}, 'corine')
folium.LayerControl().add_to(map)
#map.setOptions('HYBRID') # To see GE map underneath

map.add_child(folium.LatLngPopup())
map



In [59]:
# Load a pre-computed Landsat composite for input.
#input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')
#training_image = S2.filterBounds(ee.Geometry.Point(-0.892, 41.650)).sort('CLOUDY_PIXEL_PERCENTAGE').first()

# Define a region in which to generate a sample of the input.
region = ee.Geometry.Rectangle(-1.29, 41.4705, -0.49, 42.3948)

# Display the sample region.

map = folium.Map(location=[41.6899,0.4561],zoom_start=8)
map.addLayer(training_image,{'min':0,'max':2000,'bands':['B4','B3','B2']},'train_img')
map.addLayer(ee.Image().paint(region, 0, 2), {}, 'region')

map.add_child(folium.LatLngPopup())
map

In [60]:
# Make the training dataset.
training = training_image.sample(region = region,scale= 30,  numPixels= 5000)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Cluster the input using the trained clusterer.
result = training_image.cluster(clusterer)

# Display the clusters with random colors.

map.addLayer(result.randomVisualizer(), {}, 'clusters')
folium.LayerControl().add_to(map)
map.add_child(folium.LatLngPopup())
map


In [65]:
roi = ee.Geometry.Rectangle([ 2.16318821,41.39104253,2.19564576,41.40965281])

# Make the training dataset.
training = mosaic.sample(region = roi,scale= 30,  numPixels= 5000)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(8).train(training)

# Cluster the input using the trained clusterer.
result = mosaic.clip(roi).cluster(clusterer)

# Display the clusters with random colors.

map = folium.Map(location=[41.4003477,2.17941699],zoom_start=14.9)
map.addLayer(mosaic.clip(roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'sen2')
map.addLayer(result.randomVisualizer(), {}, 'clusters')
folium.LayerControl().add_to(map)
map.add_child(folium.LatLngPopup())
map


In [68]:
roi = ee.Geometry.Rectangle([ 2.16318821,41.39104253,2.19564576,41.40965281])

# Make the training dataset.
training = mosaic.sample(region = roi,scale= 30,  numPixels= 5000)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(5).train(training)

# Cluster the input using the trained clusterer.
result = mosaic.clip(roi).cluster(clusterer)

# Display the clusters with random colors.

map = folium.Map(location=[41.4003477,2.17941699],zoom_start=14.9)
map.addLayer(mosaic.clip(roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'sen2')
map.addLayer(result.randomVisualizer(), {}, 'clusters')
folium.LayerControl().add_to(map)
map.add_child(folium.LatLngPopup())
map
